In [1]:
%%capture
!pip install optuna

In [ ]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Basic Hyperparameter Tuning

In [7]:
import torch

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = [0 if label == 0 else 1 for label in df['HeartDisease']]
        self.features = df.drop(columns=['HeartDisease'], axis=1).values.tolist()

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_features(self, idx):
        return np.array(self.features[idx])

    def __getitem__(self, idx):
        batch_features = self.get_batch_features(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_features, batch_y

In [8]:

# Build neural network model
def build_model(params):
    
    in_features = 20
    
    return nn.Sequential(
    
        nn.Linear(in_features, params['n_unit']),
        nn.LeakyReLU(),
        nn.Linear(params['n_unit'], 2),
        nn.LeakyReLU()
        
    )
 
 
# Train and evaluate the accuarcy of neural network model
def train_and_evaluate(param, model):
    
    df = pd.read_csv('heart.csv')
    df = pd.get_dummies(df)
    
    train_data, val_data = train_test_split(df, test_size = 0.2, random_state = 42)
    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr= param['learning_rate'])

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(EPOCHS):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in train_dataloader:

                train_label = train_label.to(device)
                train_input = train_input.to(device)

                output = model(train_input.float())
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    val_input = val_input.to(device)

                    output = model(val_input.float())

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            accuracy = total_acc_val/len(val_data)

    return accuracy
  
 # Define a set of hyperparameter values, build the model, train the model, and evaluate the accuracy 
def objective(trial):

     params = {
              'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
              'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              'n_unit': trial.suggest_int("n_unit", 4, 18)
              }
    
     model = build_model(params)
    
     accuracy = train_and_evaluate(params, model)

     return accuracy

In [9]:
EPOCHS = 30
    
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=30)

[I 2021-12-13 16:56:10,183] A new study created in memory with name: no-name-e6be42b7-e2dd-4c4b-ba5c-910a833cba5d
[I 2021-12-13 16:56:18,121] Trial 0 finished with value: 0.8532608695652174 and parameters: {'learning_rate': 0.00038590927129643587, 'optimizer': 'Adam', 'n_unit': 18}. Best is trial 0 with value: 0.8532608695652174.
[I 2021-12-13 16:56:23,990] Trial 1 finished with value: 0.7119565217391305 and parameters: {'learning_rate': 3.444215185938194e-05, 'optimizer': 'SGD', 'n_unit': 12}. Best is trial 0 with value: 0.8532608695652174.
[I 2021-12-13 16:56:31,247] Trial 2 finished with value: 0.5652173913043478 and parameters: {'learning_rate': 9.230694800141404e-05, 'optimizer': 'RMSprop', 'n_unit': 7}. Best is trial 0 with value: 0.8532608695652174.
[I 2021-12-13 16:56:38,413] Trial 3 finished with value: 0.8695652173913043 and parameters: {'learning_rate': 0.0017726388721007117, 'optimizer': 'RMSprop', 'n_unit': 14}. Best is trial 3 with value: 0.8695652173913043.
[I 2021-12-13

In [14]:
best_trial = study.best_trial

for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

learning_rate: 0.005177263381010563
optimizer: RMSprop
n_unit: 15


# Hyperparameter Tuning with Pruning and Define-by-Run Design

In [15]:
# Build a model by implementing define-by-run design from Optuna
def build_model_custom(trial):
    
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 20
    
    for i in range(n_layers):
        
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 18)
        
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.LeakyReLU())

        in_features = out_features
        
    layers.append(nn.Linear(in_features, 2))
    layers.append(nn.LeakyReLU())
    
    return nn.Sequential(*layers)

# Train and evaluate the accuracy of neural network with the addition of pruning mechanism
def train_and_evaluate(param, model, trial):
    
    df = pd.read_csv('heart.csv')
    df = pd.get_dummies(df)
    
    train_data, val_data = train_test_split(df, test_size = 0.2, random_state = 42)
    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr= param['learning_rate'])

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(EPOCHS):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in train_dataloader:

                train_label = train_label.to(device)
                train_input = train_input.to(device)

                output = model(train_input.float())
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    val_input = val_input.to(device)

                    output = model(val_input.float())

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            accuracy = total_acc_val/len(val_data)
            
            # Add prune mechanism
            trial.report(accuracy, epoch_num)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    return accuracy
  
# Define a set of hyperparameter values, build the model, train the model, and evaluate the accuracy
def objective(trial):

     params = {
              'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
              'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              }
    
     model = build_model_custom(trial)

     accuracy = train_and_evaluate(params, model, trial)

     return accuracy
  
  
EPOCHS = 30
    
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=30)

[I 2021-12-13 17:06:47,385] A new study created in memory with name: no-name-c1e0e883-6e38-496f-9269-6da4430fd575
[I 2021-12-13 17:06:52,831] Trial 0 finished with value: 0.6902173913043478 and parameters: {'learning_rate': 0.000541142631699109, 'optimizer': 'SGD', 'n_layers': 1, 'n_units_l0': 8}. Best is trial 0 with value: 0.6902173913043478.
[I 2021-12-13 17:07:00,616] Trial 1 finished with value: 0.8043478260869565 and parameters: {'learning_rate': 0.043736013089081996, 'optimizer': 'Adam', 'n_layers': 1, 'n_units_l0': 7}. Best is trial 1 with value: 0.8043478260869565.
[I 2021-12-13 17:07:07,788] Trial 2 finished with value: 0.6467391304347826 and parameters: {'learning_rate': 0.0008451663395897186, 'optimizer': 'SGD', 'n_layers': 2, 'n_units_l0': 12, 'n_units_l1': 12}. Best is trial 1 with value: 0.8043478260869565.
[I 2021-12-13 17:07:15,370] Trial 3 finished with value: 0.5706521739130435 and parameters: {'learning_rate': 3.235228507760018e-05, 'optimizer': 'Adam', 'n_layers': 

In [16]:
best_trial = study.best_trial

for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

learning_rate: 0.0007651295805457043
optimizer: RMSprop
n_layers: 1
n_units_l0: 16


# Tuning Visualization

In [17]:
optuna.visualization.plot_intermediate_values(study)

In [18]:
optuna.visualization.plot_optimization_history(study)

In [19]:
optuna.visualization.plot_parallel_coordinate(study)

In [20]:
optuna.visualization.plot_param_importances(study)